In [3]:
# ETL australian_users_items.json

import json
import ast
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

user_items = []

# Abre el archivo y recorrerlo para agregar los items a la lista
with open('../data_sources/json/australian_users_items.json', encoding='utf-8') as f:
	for line in f:
		object = json.loads(json.dumps(ast.literal_eval(line)))
		user_items.append(object)

df_user_items = pd.DataFrame(user_items)

# Normalizar el campo items
normalized = pd.json_normalize(user_items, record_path=['items'], meta=['user_id'] )

# Eliminar duplicados
clean_items = normalized.drop_duplicates(keep='first')
clean_items = clean_items.drop(columns=['playtime_2weeks'])

# Convierte el DataFrame de Pandas a una tabla Arrow
table = pa.Table.from_pandas(clean_items)

# Especifica la ruta del archivo Parquet
file = '../data_sources/parquet/users_items.parquet'

# Escribe la tabla Arrow en un archivo Parquet
pq.write_table(table, file, compression='gzip')